# Gated Recurrent Unit

In [34]:
# Import des Apple Datensatzes
%store -r df

In [48]:
# Import der benötigten Bibliotheken für Neuronale Netze
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [49]:
# Aufteilung in Trainings- und Testdaten
X = df.drop(columns = ["close_next"])
y = df["close_next"]

# Trainings- und Testdaten
X_train_test, X_test, y_train_test, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Trainings- und Validierungsdaten
X_train, X_val, y_train, y_val = train_test_split(X_train_test, y_train_test, test_size = 0.25, random_state = 42)

# Daten Normalisieren mithilfe des MinMaxScaler
sc = MinMaxScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)
X_val_scaled = sc.transform(X_val)

In [55]:
# Reshape der Eingabedaten, damit das GRU-Modell trainiert werden kann
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])
X_val_reshaped = X_val_scaled.reshape(X_val_scaled.shape[0], 1, X_val_scaled.shape[1])

### GRU-Modell:

In [78]:
# Funktion zur Erstellung des Keras Modells
def create_model():
    model = Sequential()
    
    #Erste GRU-Schicht
    model.add(GRU(units = 256, return_sequences = True, input_shape = (1, 7)))
    model.add(Dropout(0.2))
    
    #Zweite GRU-Schicht
    model.add(GRU(units = 512, return_sequences = True))
    model.add(Dropout(0.2))
    
    #dritte GRU-Schicht
    model.add(GRU(units = 512, return_sequences = True))
    model.add(Dropout(0.2))

    #vierte GRU-Schicht
    model.add(GRU(units = 256))
    model.add(Dropout(0.2))
    
    #Dense-Schichten
    model.add(Dense(units = 64, activation = "relu", kernel_regularizer = regularizers.l2(0.05)))
    model.add(Dense(units = 32, activation = "relu", kernel_regularizer = regularizers.l2(0.05)))
    model.add(Dense(units = 16, activation = "relu", kernel_regularizer = regularizers.l2(0.05)))
    model.add(Dense(units = 8, activation = "relu", kernel_regularizer = regularizers.l2(0.05)))
    model.add(Dense(units = 1))
    
    model.compile(loss = "mean_squared_error", optimizer = "adam")
    return model

# Erstellen eines KeraRegressor
gru = KerasRegressor(model = create_model, verbose = 0)

# Fit
gru.fit(X_train_reshaped, y_train, epochs = 100, batch_size = 32)

# Bewertung auf Basis der Testdaten
gru_test_score = gru.score(X_test_reshaped, y_test)
print("test R^2 score Gated Recurrent Unit: {:.2f}%".format(gru_test_score * 100))

# Vorhersage auf Basis der Testdaten
y_pred = gru.predict(X_test_reshaped)

# mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)

# mean_squared_error
mse = mean_squared_error(y_test, y_pred)

# Root mean squared error
rmse = np.sqrt(mse)

# mean absolute percentage error
mape = mean_absolute_percentage_error(y_test, y_pred)

print("Mean Absolute Error (MAE) Gated Recurrent Unit: {:.2f}".format(mae))
print("Mean Squared Error (MSE) Gated Recurrent Unit: {:.2f}".format(mse))
print("Root Mean Squared Error (RMSE) Gated Recurrent Unit: {:.2f}".format(rmse))
print("Mean Absolute Percentage Error (MAPE) Gated Recurrent Unit: {:.2f}".format(mape))

test R^2 score Gated Recurrent Unit: 99.93%
Mean Absolute Error (MAE) Gated Recurrent Unit: 0.55
Mean Squared Error (MSE) Gated Recurrent Unit: 1.07
Root Mean Squared Error (RMSE) Gated Recurrent Unit: 1.03
Mean Absolute Percentage Error (MAPE) Gated Recurrent Unit: 0.44
